In [5]:
import os
import json
import numpy as np

from tqdm import tqdm
from PIL import Image
from blenderproc.python.writer import CocoWriterUtility


In [25]:
# data_path = "/home/yangfei/clean-pvnet/data/FIT/insert_mold_256_test" # replace with the path to your dataset
data_path = "/home/yangfei/clean-pvnet/data/FIT/insert_mold_256" # replace with the path to your dataset
coco_path = data_path + "/coco_annotations.json"
with open(coco_path, 'r') as file:
    coco_anno = json.load(file)

print(coco_anno['info'])
print(coco_anno['licenses'])
print(coco_anno['images'])
print(coco_anno['annotations'])
print(coco_anno['categories'])
print("Number of data: ", len(coco_anno['annotations']))

{'description': 'coco_annotations', 'url': 'https://github.com/waspinator/pycococreator', 'version': '0.1.0', 'year': 2020, 'contributor': 'Unknown', 'date_created': '2023-06-08 20:34:30.965496'}
[{'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License', 'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/'}]
[{'id': 0, 'file_name': 'images/000000.png', 'width': 256, 'height': 256, 'date_captured': '2023-06-08 20:34:30.965506', 'license': 1, 'coco_url': '', 'flickr_url': ''}, {'id': 1, 'file_name': 'images/000001.png', 'width': 256, 'height': 256, 'date_captured': '2023-06-08 20:34:33.644518', 'license': 1, 'coco_url': '', 'flickr_url': ''}, {'id': 2, 'file_name': 'images/000002.png', 'width': 256, 'height': 256, 'date_captured': '2023-06-08 20:34:36.311576', 'license': 1, 'coco_url': '', 'flickr_url': ''}, {'id': 3, 'file_name': 'images/000003.png', 'width': 256, 'height': 256, 'date_captured': '2023-06-08 20:34:39.023903', 'license': 1, 'coco_url': '', 'flickr_url': ''},

### Parse coco annotation and generate binary mask images

In [26]:
annotations = coco_anno['annotations']
for annotation in annotations:
    # print(annotation)
    image_id = annotation['image_id']
    rle = annotation['segmentation']
    binary_mask = CocoWriterUtility.rle_to_binary_mask(rle)
    binary_mask = np.array(binary_mask, dtype=np.uint8)*255
    
    # Convert the binary mask array to grayscale image
    mask_image_gray = Image.fromarray(binary_mask)

    # Convert the grayscale image to RGB
    mask_image_rgb = mask_image_gray.convert("RGB")

    # Save the RGB mask image
    mask_image_rgb.save(data_path + f"/mask/{image_id}.png")

### Rename images and folder

In [27]:
image_path = os.path.join(data_path, "images")

# Get the list of files in the folder
files = os.listdir(image_path)

# Iterate over the files and rename them with tqdm progress bar
for i, filename in tqdm(enumerate(files), total=len(files), desc="Renaming files"):
    # Generate the new filename
    new_filename = f"{int(filename.split('.')[0])}.png"
    # print(new_filename)
    
    # Get the full paths of the current and new filenames
    current_path = os.path.join(image_path, filename)
    new_path = os.path.join(image_path, new_filename)
    
    # Rename the file
    os.rename(current_path, new_path)


# Create the new path with the new folder name
new_folder_path = os.path.join(data_path, "rgb")
# Rename the folder
os.rename(image_path, new_folder_path)

Renaming files: 100%|██████████| 500/500 [00:00<00:00, 37916.33it/s]
